# Reinforcement Learning
Prof. S. Harmeling, SS 2024

### Exercise sheet #4

#### 2. Implement Monte Carlo prediction.

The idea of Monte Carlo prediction is very simple: Estimate the value (or the action value) by averating the observed returns from collected episodes. In this notebook we apply Monte Carlo prediction to the game of tic-tac-toe.

#### Implementation

Make sure that the file `rl_env.py` is in the same folder as the notebook (the same file as in exercise sheet #2).

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import rl_env

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We already implemented tic-tac-toe in `TicTacToeEnv`:
- The environment has $3^9 = 19683$ states (9 fields with 3 values: empty, player 1, player 2).
- There are $9$ possible actions, which determine the next move of the current player (i.e. the actions control both players).
- The final reward is $1$ if player 1 wins, and $0$ if player 2 wins or when there is a draw. The reward is $0$ in all other time steps.

In [5]:
# Create an instance of the tic-tac-toe environment
env = rl_env.TicTacToeEnv()

We already implemented the random policy for the tic-tac-toe environment:

In [6]:
def random_policy(state):
    # Obtain the list of empty fields
    valid_actions = rl_env.TicTacToeEnv.get_valid_actions(state)
    # Select one of the empty fields randomly
    # For non-empty fields, the action does not have an effect
    action = np.random.choice(valid_actions)
    return action

Your task is to implement Monte Carlo prediction of the action value for the **initial state**, i.e. you don't need to compute the action values for all states, but only the $9$ action values for the initial state.  
We don't need a discount factor, so the initial return is equal to the final reward.

You don't need an `Agent` object for this implementation, just generate episodes and estimate the action values.

In [7]:
#######################################################################
# TODO: Implement Monte Carlo prediction of the action value function #
# for the initial state as described above. Generate at least 10000   #
# episodes to estimate the action values.                             #
#######################################################################
# Create arrays for total returns and counts for the initial state
G = np.zeros(9, dtype=float)
N = np.zeros(9, dtype=int)

# Generate 10000 episodes
for i in range(10000):
    # Reset the environment
    state, _ = env.reset()
    # Remember the start action to update the action value
    start_action = random_policy(state)

    # Rollout the episode
    action = start_action
    while True:
        state, reward, terminated, truncated, _ = env.step(action)
        if terminated or truncated:
            break
        action = random_policy(state)

    if not truncated:
        # We cannot use the episode if it was truncated, since we cannot
        # compute the true returns (not important for this environment)

        # No rewards until the end and no discounting,
        # so the initial return is equal to final reward
        g = reward
        # Update the total return and count for the initial state
        G[start_action] += g
        N[start_action] += 1

# Compute the action values
q = G / N


#######################################################################
# End of your code.                                                   #
#######################################################################

In [8]:
q

array([0.61717352, 0.5328084 , 0.60154242, 0.55996223, 0.68726937,
       0.53946147, 0.62697674, 0.53953084, 0.60834813])

Since the reward is only $1$ if player 1 wins, the value of the initial state is equal to the winning probability of player 1.  
Use this to answer the following questions:
- What is the probability that the first player wins?
- Which initial action has the highest chance of winning?

In [9]:
#######################################################################
# TODO: Answer the questions by using the computed action values.     #
#######################################################################
# Convert the action values to state values
# Since all actions in the initial state have uniform probability (1/9),
# we can just calculate the mean
# v(s) = \sum_a \pi(a|s) q(s,a) = 1/9 * \sum_a q(s,a)
v = q.mean()

print('total returns:', *[f'{x:.1f}' for x in G])
print('counts:', *N)
print('q:', *[f'{x:.3f}' for x in q])
print('v:', f'{v:.3f}')
print('average win chance:', f'{v * 100:.2f}%')
print('best start action:', np.argmax(q))
print('win chance of best start action:', f'{np.max(q) * 100:.2f}%')

# Action 4 is the field in the middle of the board
#######################################################################
# End of your code.                                                   #
#######################################################################

total returns: 690.0 609.0 702.0 593.0 745.0 581.0 674.0 621.0 685.0
counts: 1118 1143 1167 1059 1084 1077 1075 1151 1126
q: 0.617 0.533 0.602 0.560 0.687 0.539 0.627 0.540 0.608
v: 0.590
average win chance: 59.03%
best start action: 4
win chance of best start action: 68.73%
